In [10]:
%pip install mpld3
%pip install bokeh



You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [39]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource
from PIL import Image
import base64

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return "data:image/png;base64," + base64.b64encode(image_file.read()).decode('utf-8')

def generate_plot(percentages, radii,  images, sample_name):
    # Create a dictionary to map sample_name to colors
    unique_sample_names = list(set(sample_name))
    colors = ['green', 'red']  # Add more colors if you have more sample names
    color_dict = {name: colors[i % len(colors)] for i, name in enumerate(unique_sample_names)}
    # Use color_dict to get corresponding color for each sample name
    colors = [color_dict[name] for name in sample_name]

    x = radii
    y = percentages

    source = ColumnDataSource(data=dict(
        x=x,
        y=y,
        images=images,
        percentages=percentages,
        sample_name=sample_name,
        color=colors  # Add color to source data
    ))

    p = figure(title="hirox result", width=600, height=400, x_axis_label='diameter', y_axis_label='mixing zone percentage')

    # Use color attribute for the color parameter
    p.circle('x', 'y', size=10, color='color', source=source)
    hover = HoverTool(tooltips="""
        <div>
            <div>
                <img
                    src="@images" height="200" alt="@percentages" width="200"
                    style="float: left; margin: 0px 15px 15px 0px;"
                ></img>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">@percentages @sample_name</span>
            </div>
        </div>
    """)

    p.add_tools(hover)
    URL=""

    tap_js = CustomJS(args=dict(source=source), code="""
    var images = source.data['images'];
    var index = source.selected.indices[0];
    var dataURL = "/Users/wangyifan/Desktop/DFLT  WEB code/templates/image4.png"
    
    console.log(dataURL); 
    var link = document.createElement('a');
    link.href = dataURL;
    link.download = 'image.png';
    link.target = '_blank';
    link.click();
    """)
    
    p.js_on_event('tap', tap_js)


    show(p)

radii = [200,200,200]
percentages = [39.4, 28, 38.8]
sample_name=["925-52-2", "925-52-2","925-52-3"]

images = [
    './templates/image4.png',
    './templates/image5.png',
    './templates/image6.png'
]

encoded_images = [image_to_base64(image_path) for image_path in images]

generate_plot(percentages, radii, encoded_images, sample_name)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# these are hypothetical data points, replace them with your actual data
x = np.array([200, 250, 300])  # Liquidus Temperature (°C)
y = np.array([198, 200, 350])  # Drop Shock Avg

plt.figure(figsize=(10, 6))
plt.plot(x, y, 'o-')
plt.title('Liquidus Temperature vs Drop Shock Avg')
plt.xlabel('Liquidus Temperature (°C)')
plt.ylabel('Drop Shock Avg')
plt.grid(True)
plt.show()


In [45]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

# 创建一个Figure对象
p = figure()

# 添加一个数据点
p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5])

# 创建一个HoverTool对象，设置mode参数为"mouse"
hover = HoverTool(tooltips=[("x", "@x"), ("y", "@y")], mode="mouse")

# 将HoverTool对象添加到Figure对象中
p.add_tools(hover)

# 显示图表
show(p)


In [ ]:
import requests

# You should replace these with your own details
CLIENT_ID = "your_client_id"
CLIENT_SECRET = "your_client_secret"
REDIRECT_URI = "http://localhost"
AUTHORITY_URL = "https://login.microsoftonline.com/common"
AUTH_ENDPOINT = '/oauth2/v2.0/authorize'
TOKEN_ENDPOINT = '/oauth2/v2.0/token'
SCOPES = ['Files.ReadWrite', 'User.Read', 'openid', 'profile', 'offline_access']

# function to authenticate user and get access token
def get_access_token(client_id, client_secret, redirect_uri, authority_url, scopes):
    # build the url
    url = authority_url + TOKEN_ENDPOINT

    # build the header
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    # build the payload (data)
    payload = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': ' '.join(str(i) for i in scopes)
    }

    # send the request
    r = requests.post(url, headers=headers, data=payload)

    # return the access token
    return r.json().get('access_token')

# function to get files from onedrive
def get_onedrive_files(access_token):
    # endpoint to get the files
    endpoint = "https://graph.microsoft.com/v1.0/me/drive/root/children"

    # headers
    headers = {
        'Authorization': 'Bearer ' + access_token
    }

    # send the request
    r = requests.get(endpoint, headers=headers)

    # print the names of the files
    for file in r.json().get('value', []):
        print(file.get('name'))

def main():
    access_token = get_access_token(CLIENT_ID, CLIENT_SECRET, REDIRECT_URI, AUTHORITY_URL, SCOPES)
    get_onedrive_files(access_token)

# Run the main function
if __name__ == "__main__":
    main()
